In [1]:
import scipy.spatial as spatial
import numpy as np
import matplotlib.pyplot as plt
from more_itertools import powerset
import mpl_toolkits.mplot3d as a3
from wireframes import wireframe_sphere
from rich import print
import gudhi as gd
from more_itertools import powerset
import time

In [2]:
X = np.asarray([
    [0,0],
    [0.5, 0],
    [0.25, 0.5 * np.sqrt(3)/2 ],
    [ 0.75/3, (0.5 * np.sqrt(3)/2)/3 ],
])
X = np.random.rand(4,3)
X = np.random.rand(10, 3)
dimension = X.shape[1]

In [3]:
def cayley_menger_matrix(X):
    d = X.shape[1]
    dm = np.power(spatial.distance_matrix(X,X), 2)
    m = np.zeros(np.asarray(dm.shape) + 1)
    m[:,0] = 1
    m[0,:] = 1
    m[0,0] = 0
    m[1:,1:] = dm
    return m

The Cartesian coordinates of any point are a weighted average
of the Cartesian coordinates of the triangle's vertices,
with the weights being the point's barycentric coordinates summing to unity. So e.g. for a triangle:


In [5]:
%matplotlib notebook
start_time = time.time()
#----------------------------------------------------------
if True:
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(projection = "3d") if dimension == 3 else fig.add_subplot()
else:
    ax = None

tri = list() if dimension == 3 else None
kdtree = spatial.KDTree(X) # X is in R^d
cech_complex = dict()

triangulation = spatial.Delaunay(X)
delaunay_complex = [ list(powerset(face)) for face in triangulation.simplices ]
delaunay_complex = set([ tuple(sorted(list(item))) for sublist in delaunay_complex for item in sublist])

if dimension == 3:
    for face in delaunay_complex:
        if len(face) == 3:
            tri.append(face)

filtration = { (i,) : 0.0 for i in range(X.shape[0]) }

for i in reversed(range(1, dimension + 1)):
    faces = [ f for f in delaunay_complex if len(f) == (i + 1) ]
    for f in faces:
        if f not in filtration:
            content_inv  = np.linalg.inv(cayley_menger_matrix( X[f,:] ))
            circumradius = np.sqrt(content_inv[0,0]/-2)
            circumcentre = content_inv[1:,0].dot( X[f,:] )
            filtration[f] = circumradius
        #
        subfaces = [ tuple(sorted(list(tau))) for tau in set(powerset(f)) if len(tau) > 1]
        for t in subfaces:
            if t in filtration:
                filtration[t] = min(filtration[t], filtration[f])
            else:
                content_inv  = np.linalg.inv(cayley_menger_matrix( X[t,:] ))
                circumradius = np.sqrt(content_inv[0,0]/-2)
                circumcentre = content_inv[1:,0].dot( X[t,:] )
                _,nn = kdtree.query(circumcentre, k = len(t))
                if set(nn) != set(t): # not Gabriel
                    filtration[t] = filtration[f]
            
            #---- PLOTTING
            if ax is not None:
                plot_color = { 2 : "black", 3 : "red", 4 : "blue" }
                plot_color = plot_color[len(t)]
                ax.scatter(
                    *tuple(circumcentre),
                    color = plot_color,
                    marker = "o",
                    facecolors='none',
                    s = 100,
                    alpha = 1
                )
                for y in X[t, :]:
                    connecting_lines = (
                        [circumcentre[0],y[0]],
                        [circumcentre[1],y[1]],
                        [circumcentre[2],y[2]]
                    ) if dimension == 3 else (
                        [circumcentre[0],y[0]],
                        [circumcentre[1],y[1]],
                    )
                    ax.plot(
                        *connecting_lines,
                        "--",
                        color = plot_color,
                        alpha = 0.4
                    )
                _,nn = kdtree.query(circumcentre, k = len(t))
                if set(nn) != set(t):
                    ax.scatter(
                        *tuple(circumcentre),
                        color = plot_color,
                        marker = "x",
                        s = 100,
                        alpha = 1
                    )
#

cech_complex = sorted(list(filtration.items()), key = lambda t : t[1])
#----------------------------------------------------------
print("--- %s seconds ---" % (time.time() - start_time))

if ax is not None:
    if dimension == 3:
        plt_tri = a3.art3d.Poly3DCollection(X[tri, :])
        plt_tri.set_alpha(0.1)
        plt_tri.set_color('grey')
        ax.add_collection3d(plt_tri)

    for i,x in enumerate(X):
        ax.scatter(*tuple(x), color = "blue")
        ax.text(*tuple(x+0.015), s=str(i))

    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    ax.set_zlim(0,1) if dimension == 3 else None
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z") if dimension == 3 else None
    plt.show()

<IPython.core.display.Javascript object>

--- 1.6004643440246582 seconds ---

In [6]:
start_time = time.time()
#----------------------------------------------------------
skeleton = gd.AlphaComplex(
    points = X
)
st = skeleton.create_simplex_tree()
gudhi_cech_complex = sorted(list(map(lambda t : (t[0],np.sqrt(t[1])), st.get_filtration())), key = lambda t : t[1])
#----------------------------------------------------------
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0006544589996337891 seconds ---

In [7]:
my_values = [ t[1] for t in cech_complex ]
gd_values = [ t[1] for t in gudhi_cech_complex ]
vals = list(zip(my_values, gd_values))
checker = list(map(lambda v : np.abs(v[0] - v[1]), vals))
close_val_check = np.allclose(checker, 0)
print('Filtration values are all close: ' + str(close_val_check))
if not close_val_check:
    a = np.asarray(checker)
    c = np.logical_not(np.isclose(checker, 0))
    s = sum([1 for x in c if x])
    print(str(s)+' filtration values were different: ')
    print(a[c])

Filtration values are all close: True

In [13]:
my_simplices = [ X[t[0], :] for t in cech_complex ]
gd_simplices = [ np.asarray([skeleton.get_point(p) for p in t[0] ]) for t in gudhi_cech_complex ]

def same_face(a, b):
    if len(a) != len(b):
        return False
    return np.allclose(np.abs(a - b), 0)

for s in my_simplices:
    print(any(list(map(lambda f : same_face(f,s), gd_simplices ))))

True

True

True

True

True

...

True

True

True

True

ValueError: operands could not be broadcast together with shapes (3,3) (2,3) 